In [1]:
import torch
import torch.nn as nn
import torch.onnx
import torchvision

MODEL_PATH = "/home/namdng/garbage_classifier/models/init/ckpt_26_0.9547.pth"
IMAGE_SIZE = 394
BATCH_SIZE = 2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Load model
class_names = ["cardboard_paper", "glass", "metal", "others", "plastic"]
model_conv = torchvision.models.efficientnet_v2_s()
num_ftrs = model_conv.classifier[1].in_features
model_conv.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, len(class_names))
)
model_conv = model_conv.to(device)
model_conv.load_state_dict(torch.load(MODEL_PATH, weights_only=True))
model_conv.eval()

# Input to the model
x = torch.randn(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE, device=device)
torch_out = model_conv(x)
print(torch_out)

# Export the model
torch.onnx.export(model_conv,                # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "gc.onnx",                 # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})



tensor([[-2.1735, -0.7517, -4.0516, -2.6440, -7.7478],
        [-0.4326, -1.7334, -2.2905, -5.4048, -7.2545]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


/data/anaconda3/envs/vision/lib/python3.11/site-packages/torch/onnx/utils.py:1504: OnnxExporterWarning: Exporting to ONNX opset version 21 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 20. To use a newer opset version, consider 'torch.onnx.export(..., dynamo=True)'. 
  warnings.warn(


ValueError: Unsupported ONNX opset version: 21

In [5]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("gc.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

print(to_numpy(torch_out), ort_outs[0])

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

[[-2.5980191 -0.9685682 -3.5548716 -1.5928451 -8.014211 ]
 [-1.1870288 -0.6295426 -3.2320693 -3.894133  -6.602029 ]] [[-2.5980237 -0.9685674 -3.5548658 -1.5928464 -8.014215 ]
 [-1.1870263 -0.6295472 -3.2320774 -3.8941274 -6.602043 ]]
Exported model has been tested with ONNXRuntime, and the result looks good!


### convert test images to RGB

In [1]:
import os
from PIL import Image

DATA_DIR = "data_split/test"

for class_name in os.listdir(DATA_DIR):
    class_dir = os.path.join(DATA_DIR, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = Image.open(image_path)
        image_mode = image.mode
        if image_mode != "RGB":
            image = image.convert("RGB")
            image.save(image_path)